In [1]:
import pandas as pd
import string
import re
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dgdev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dgdev\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dgdev\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv("train.csv")
print(df.shape)
df.head()

(27481, 4)


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [3]:
df.dropna(inplace=True)
df.shape

(27480, 4)

In [4]:
df.drop(columns=["textID", "selected_text"], axis=1, inplace=True)
df.shape

(27480, 2)

In [5]:
class_map = {"negative":0, "neutral":1, "positive":2}
df["sentiment"] = df["sentiment"].map(class_map)
df.head(8)

,text,sentiment
0,"I`d have responded, if I were going",1
1,Sooo SAD I will miss you here in San Diego!!!,0
2,my boss is bullying me...,0
3,what interview! leave me alone,0
4,"Sons of ****, why couldn`t they put them on t...",0
5,http://www.dothebouncy.com/smf - some shameles...,1
6,2am feedings for the baby are fun when he is a...,2
7,Soooo high,1


In [6]:
def caseLower(text) :
    text = text.lower()
    return text

def punctuationRemove(text) :
    text = "".join([char for char in text if char not in string.punctuation])
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub("[0-9]+", "", text)
    return text

def tokenization(text) :
    text = text.strip()
    text = re.split("\W+", text)
    return text

def singlesRemove(text) :
    text = [word for word in text if len(word) > 1]
    return text

def stopwordsRemove(text) :
    stopword = nltk.corpus.stopwords.words("english")
    text = [word for word in text if word not in stopword]
    return text

def lemmatizer(text) :
    text = [nltk.WordNetLemmatizer().lemmatize(word) for word in text]
    return text

def preprocess(text) :
    text = caseLower(text)
    text = punctuationRemove(text)
    text = tokenization(text)
    text = singlesRemove(text)
    text = stopwordsRemove(text)
    text = lemmatizer(text)
    text = " ".join(text)
    return text

In [12]:
df["text"] = df["text"].apply(lambda x: preprocess(x))
df.head(8)

,text,sentiment
0,id responded going,1
1,sooo sad miss san diego,0
2,bos bullying,0
3,interview leave alone,0
4,son couldnt put release already bought,0
5,httpwwwdothebouncycomsmf shameless plugging be...,1
6,feeding baby fun smile coo,2
7,soooo high,1


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["sentiment"], test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(21984,)
(5496,)
(21984,)
(5496,)


In [9]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [13]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X_train.toarray())

# Create a scatter plot
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y_train)
plt.show()

In [10]:
model = MultinomialNB()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [11]:
print(confusion_matrix(y_test, predictions))
print("------------------------------")
print(classification_report(y_test, predictions))

[[ 617  876   79]
 [ 143 1854  239]
 [  35  682  971]]
------------------------------
              precision    recall  f1-score   support

           0       0.78      0.39      0.52      1572
           1       0.54      0.83      0.66      2236
           2       0.75      0.58      0.65      1688

    accuracy                           0.63      5496
   macro avg       0.69      0.60      0.61      5496
weighted avg       0.67      0.63      0.62      5496

